## Imports 

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [24]:
from fastaidev.basics import *
from fastaidev.tabular import *

## Paths

In [3]:
PATH = Config.data_path()/'competitions/elo-merchant-category-recommendation/'
list(PATH.glob("*.csv"))

[PosixPath('/home/achen/.fastai/data/competitions/elo-merchant-category-recommendation/historical_transactions.csv'),
 PosixPath('/home/achen/.fastai/data/competitions/elo-merchant-category-recommendation/merchants.csv'),
 PosixPath('/home/achen/.fastai/data/competitions/elo-merchant-category-recommendation/new_merchant_transactions.csv'),
 PosixPath('/home/achen/.fastai/data/competitions/elo-merchant-category-recommendation/test.csv'),
 PosixPath('/home/achen/.fastai/data/competitions/elo-merchant-category-recommendation/train.csv')]

In [4]:
train = pd.read_csv(PATH/"train.csv", parse_dates=["first_active_month"])
test  = pd.read_csv(PATH/"test.csv", parse_dates=["first_active_month"])

In [5]:
_old = pd.read_csv(PATH/"historical_transactions.csv", parse_dates=['purchase_date'])

In [6]:
_new = pd.read_csv(PATH/"new_merchant_transactions.csv", parse_dates=['purchase_date'])

In [7]:
_merchant = pd.read_csv(PATH/"merchants.csv")

## What data looks like...

In [8]:
train.head().T

,0,1,2,3,4
first_active_month,2017-06-01 00:00:00,2017-01-01 00:00:00,2016-08-01 00:00:00,2017-09-01 00:00:00,2017-11-01 00:00:00
card_id,C_ID_92a2005557,C_ID_3d0044924f,C_ID_d639edf6cd,C_ID_186d6a6901,C_ID_cdbd2c0db2
feature_1,5,4,2,4,1
feature_2,2,1,2,3,3
feature_3,1,0,0,0,0
target,-0.820283,0.392913,0.688056,0.142495,-0.159749


In [9]:
test.head().T

,0,1,2,3,4
first_active_month,2017-04-01 00:00:00,2017-01-01 00:00:00,2017-08-01 00:00:00,2017-12-01 00:00:00,2015-12-01 00:00:00
card_id,C_ID_0ab67a22ab,C_ID_130fd0cbdd,C_ID_b709037bc5,C_ID_d27d835a9f,C_ID_2b5e3df5c2
feature_1,3,2,5,2,5
feature_2,3,3,1,1,1
feature_3,1,0,1,0,1


In [10]:
_old.head().T

,0,1,2,3,4
authorized_flag,Y,Y,Y,Y,Y
card_id,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc
city_id,88,88,88,88,88
category_1,N,N,N,N,N
installments,0,0,0,0,0
category_3,A,A,A,A,A
merchant_category_id,80,367,80,560,80
merchant_id,M_ID_e020e9b302,M_ID_86ec983688,M_ID_979ed661fc,M_ID_e6d5ae8ea6,M_ID_e020e9b302
month_lag,-8,-7,-6,-5,-11
purchase_amount,-0.703331,-0.733128,-0.720386,-0.735352,-0.722865


In [11]:
_new.head().T

,0,1,2,3,4
authorized_flag,Y,Y,Y,Y,Y
card_id,C_ID_415bb3a509,C_ID_415bb3a509,C_ID_415bb3a509,C_ID_415bb3a509,C_ID_ef55cf8d4b
city_id,107,140,330,-1,-1
category_1,N,N,N,Y,Y
installments,1,1,1,1,1
category_3,B,B,B,B,B
merchant_category_id,307,307,507,661,166
merchant_id,M_ID_b0c793002c,M_ID_88920c89e8,M_ID_ad5237ef6b,M_ID_9e84cda3b1,M_ID_3c86fa3831
month_lag,1,1,2,1,1
purchase_amount,-0.557574,-0.56958,-0.551037,-0.671925,-0.659904


In [12]:
_merchant.head().T

,0,1,2,3,4
merchant_id,M_ID_838061e48c,M_ID_9339d880ad,M_ID_e726bbae1e,M_ID_a70e9c5f81,M_ID_64456c37ce
merchant_group_id,8353,3184,447,5026,2228
merchant_category_id,792,840,690,792,222
subsector_id,9,20,1,9,21
numerical_1,-0.0574706,-0.0574706,-0.0574706,-0.0574706,-0.0574706
numerical_2,-0.0574706,-0.0574706,-0.0574706,-0.0574706,-0.0574706
category_1,N,N,N,Y,Y
most_recent_sales_range,E,E,E,E,E
most_recent_purchases_range,E,E,E,E,E
avg_sales_lag3,-0.4,-0.72,-82.13,NaN,NaN


## Easy sort out for NN

In [13]:
def join_df(left, right, left_on, right_on=None, suffix='_y'):
    if right_on is None: right_on = left_on
    return left.merge(right, how='left', left_on=left_on, right_on=right_on, 
                      suffixes=("", suffix))

In [14]:
def aggregate(df, by, agg_params, prefix=None):
    df = df.groupby(by).agg(agg_params)
    df.columns = [prefix + '_'.join(col).strip() for col in df.columns.values]
    df.reset_index(inplace=True)
    return df

In [15]:
agg_params = { 'purchase_amount' : ['sum', 'mean', 'std', 'min', 'max'] }
agg_df = aggregate(_old, by="card_id", agg_params=agg_params, prefix="old_")

In [16]:
joined = join_df(train, agg_df, "card_id") 
joined_test = join_df(test, agg_df, "card_id")

In [17]:
agg_params = {'purchase_amount' : ['size', 'sum', 'mean', 'std', 'min', 'max']}
agg_df = aggregate(_new, by="card_id", agg_params=agg_params, prefix="new_")

In [19]:
joined = join_df(joined, agg_df, "card_id")
joined_test = join_df(joined_test, agg_df, "card_id")

In [20]:
joined.head().T

,0,1,2,3,4
first_active_month,2017-06-01 00:00:00,2017-01-01 00:00:00,2016-08-01 00:00:00,2017-09-01 00:00:00,2017-11-01 00:00:00
card_id,C_ID_92a2005557,C_ID_3d0044924f,C_ID_d639edf6cd,C_ID_186d6a6901,C_ID_cdbd2c0db2
feature_1,5,4,2,4,1
feature_2,2,1,2,3,3
feature_3,1,0,0,0,0
target,-0.820283,0.392913,0.688056,0.142495,-0.159749
old_purchase_amount_sum,-165.969,-210.006,-29.1674,-49.4914,-48.6877
old_purchase_amount_mean,-0.638341,-0.600018,-0.678311,-0.642745,-0.366073
old_purchase_amount_std,0.212139,0.384967,0.0873804,0.261624,1.35209
old_purchase_amount_min,-0.739395,-0.7424,-0.730138,-0.740897,-0.746156


In [21]:
joined_test.head().T

,0,1,2,3,4
first_active_month,2017-04-01 00:00:00,2017-01-01 00:00:00,2017-08-01 00:00:00,2017-12-01 00:00:00,2015-12-01 00:00:00
card_id,C_ID_0ab67a22ab,C_ID_130fd0cbdd,C_ID_b709037bc5,C_ID_d27d835a9f,C_ID_2b5e3df5c2
feature_1,3,2,5,2,5
feature_2,3,3,1,1,1
feature_3,1,0,1,0,1
old_purchase_amount_sum,-40.7337,-49.1365,4.52884,-13.6907,25.1394
old_purchase_amount_mean,-0.599025,-0.629955,0.348372,-0.526566,0.22854
old_purchase_amount_std,0.192268,0.154999,0.906547,0.219162,2.77776
old_purchase_amount_min,-0.743902,-0.731881,-0.536537,-0.731881,-0.746758
old_purchase_amount_max,0.235676,0.318817,2.52587,0.0879652,15.7823


In [25]:
add_datepart(joined, 'first_active_month', drop=True)
add_datepart(joined_test, 'first_active_month', drop=True)

In [28]:
joined.isnull().sum()

card_id                                   0
feature_1                                 0
feature_2                                 0
feature_3                                 0
target                                    0
old_purchase_amount_sum                   0
old_purchase_amount_mean                  0
old_purchase_amount_std                   0
old_purchase_amount_min                   0
old_purchase_amount_max                   0
new_purchase_amount_size              21931
new_purchase_amount_sum               21931
new_purchase_amount_mean              21931
new_purchase_amount_std               48718
new_purchase_amount_min               21931
new_purchase_amount_max               21931
first_active_monthYear                    0
first_active_monthMonth                   0
first_active_monthWeek                    0
first_active_monthDay                     0
first_active_monthDayofweek               0
first_active_monthDayofyear               0
first_active_monthIs_month_end  

In [ ]:
#scale the data and impute the null values 
#note: apparently, GPU environment doesn't have an updated version of sklearn,
#so we cannot use sklearn.impute.SimpleImputer. In CPU environement this is possible
sc = StandardScaler()
train_X = train_X.fillna(0)
train_X = sc.fit_transform(train_X)
x_train, x_val, y_train, y_val = train_test_split(train_X, train_y, test_size = .1, random_state = 42)